In [12]:
import copy
lines = open("input_20.txt").read().splitlines()
# [print(x) for x in lines]
# print(' ')

# build dictionary of instructions
cfg = {}
start_state = 0  # 0:off, 1:on
for line in lines:
    input, outputs = line.split(' -> ')
    if input[0] in '%&':
        key = input[1:]
        cfg[key] = [start_state, input[0], outputs.split(', ')]
    else:
        key = input
        cfg[key] = [start_state, '', outputs.split(', ')]
cfg_orig = copy.deepcopy(cfg)
[print(x,cfg[x]) for x in cfg]
print(' ')

# build a dictionary of conjuction inputs
conj_inputs = {}
for gate in cfg:
    if cfg[gate][1] == '&':
        inputs = []
        for gate2 in cfg:
            if gate in cfg[gate2][2]:
                inputs.append(gate2)
        conj_inputs[gate] = inputs
print('inputs =',conj_inputs)

# push the button
pulses = [0,0]  # lows, highs
for i in range(1000):
    gate = 'button'
    pulse = 'L'
    outputs = ['broadcaster']
    pulses[0] += 1
    # print(f'{gate:11} --> {pulse} --> {outputs[0]:20}pulses = {pulses}')

    todo = []
    gate = 'broadcaster'
    pulse = 'L'
    outputs = cfg[gate][2]
    pulses[0] += len(outputs)
    for a in range(len(outputs)):
        todo.append([pulse,outputs[a]])
        # print(f'{gate:11} --> {pulse} --> {outputs[a]:20}pulses = {pulses}')

    while len(todo) > 0:
        pulse,gate = todo.pop(0)
        if gate in cfg:
            state = cfg[gate][0]
            type = cfg[gate][1]
            outputs = cfg[gate][2]
        else:
            # this gate is not in the configuration, probably the output
            state = pulse
            type = '-'
            outputs = 'none'
            # print('unconfigured gate:',gate,'is in state:',state)
        if pulse == 'H' and type == '%':  # high pulse to flip-flop
            1  # do nothing
        elif pulse == 'L' and type == '%':  # low pulse to flip-flop
            if state == 0:
                cfg[gate][0] = 1
                pulse = 'H'
                pulses[1] += len(outputs)
            else:
                cfg[gate][0] = 0
                pulse = 'L'
                pulses[0] += len(outputs)

            for a in range(len(outputs)):
                todo.append([pulse,outputs[a]])
                # print(f'{gate:11} --> {pulse} --> {outputs[a]:20}pulses = {pulses}')

        # Conjuction: If it remembers high pulses for all inputs, it sends a low pulse;
        # otherwise, it sends a high pulse.
        if type == '&':  # conjunction
            inputs = conj_inputs[gate]
            states = [cfg[input][0] for input in inputs]
            if states == [1]*len(states):  # all input states are high
                pulse = 'L'
                pulses[0] += len(outputs)
            else:
                pulse = 'H'
                pulses[1] += len(outputs)

            for a in range(len(outputs)):
                todo.append([pulse,outputs[a]])
                # print(f'{gate:11} --> {pulse} --> {outputs[a]:20}pulses = {pulses}')
    print(f'button push #: {i+1}, config the same? {cfg == cfg_orig}')
    print(cfg_orig)
    print(cfg)
    print('pulses =',pulses,'lows*highs =',pulses[0]*pulses[1])
    print(' ')
    # if cfg == cfg_orig:
    #     break

    # 535448680 is too low


hm [0, '%', ['lg']]
ph [0, '%', ['gr', 'cd']]
xs [0, '%', ['st', 'zf']]
xc [0, '%', ['vv', 'bn']]
fx [0, '%', ['lx', 'st']]
dn [0, '%', ['zc']]
nd [0, '%', ['lg', 'tc']]
pd [0, '%', ['dn', 'lg']]
pc [0, '%', ['gr']]
lx [0, '%', ['gp']]
tc [0, '%', ['kj']]
tl [0, '%', ['nn']]
kk [0, '%', ['xs']]
gq [0, '%', ['pd']]
xq [0, '%', ['st', 'xj']]
nn [0, '%', ['bn', 'xc']]
st [0, '&', ['kk', 'fx', 'hz', 'lx', 'zb']]
hb [0, '&', ['rx']]
xj [0, '%', ['st', 'vq']]
sz [0, '%', ['gr', 'ph']]
bz [0, '%', ['kx']]
vq [0, '%', ['st']]
vv [0, '%', ['hh', 'bn']]
gp [0, '%', ['st', 'hz']]
js [0, '&', ['hb']]
lf [0, '%', ['bn', 'qg']]
broadcaster [0, '', ['nd', 'fx', 'mc', 'lf']]
cd [0, '%', ['vr']]
vr [0, '%', ['qc', 'gr']]
kx [0, '%', ['kc', 'lg']]
jr [0, '%', ['bn', 'tl']]
gr [0, '&', ['cz', 'dh', 'mc', 'qc', 'js', 'nj', 'cd']]
qg [0, '%', ['hq']]
mc [0, '%', ['gr', 'cz']]
nl [0, '%', ['st', 'ch']]
hz [0, '%', ['nl']]
kt [0, '%', ['gr', 'jc']]
zc [0, '%', ['lg', 'qn']]
vj [0, '%', ['rs']]
zb [0, '&', ['

In [4]:
cfg_orig2

NameError: name 'cfg_orig2' is not defined